<a href="https://colab.research.google.com/github/Jescas981/mydl-library/blob/main/examples/notebooks/MNIST_Model_784.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip uninstall mydllib -y
!pip install git+https://github.com/Jescas981/mydl-library
#!pip freeze

  Cloning https://github.com/Jescas981/mydl-library to /tmp/pip-req-build-b1m08rxd
  Running command git clone --filter=blob:none --quiet https://github.com/Jescas981/mydl-library /tmp/pip-req-build-b1m08rxd
  Resolved https://github.com/Jescas981/mydl-library to commit 794bf19ff87d21fa5813d75e183b36241a875876
  Preparing metadata (setup.py) ... done


In [2]:
from mydllib import layers, functional
from mydllib.utilities import DataLoader
import numpy as np
import torch

In [3]:
# Import data
base_url = 'http://yann.lecun.com/exdb/mnist'
loader = DataLoader(download_dir='./data',
                    urls=[
                        f'{base_url}/train-images-idx3-ubyte.gz',
                        f'{base_url}/train-labels-idx1-ubyte.gz',
                        f'{base_url}/t10k-images-idx3-ubyte.gz',
                        f'{base_url}/t10k-labels-idx1-ubyte.gz'
                        ])
train_images, train_labels, test_images, test_labels = loader.read_dataset()

In [4]:
from mydllib.functional import StandardScaler

# Normalize data
scaler = StandardScaler()
train_images_norm = scaler.fit_transform(train_images)
test_images_norm = scaler.transform(test_images)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Turn data to tensors
X_train = torch.tensor(train_images_norm, dtype=torch.float32).to(device)
X_test = torch.tensor(test_images_norm, dtype=torch.float32).to(device)
y_train = torch.tensor(train_labels, dtype=torch.long).to(device)
y_test = torch.tensor(test_labels, dtype=torch.long).to(device)

Using device: cuda


In [20]:
# Create Model
from mydllib import layers, modules
from typing import List
import torch

class MNIST784(modules.Model):
    def __init__(self, device:str) -> None:
        super().__init__(device)
        self.layers: List[layers.Layer] = [
            layers.Conv(size=(16, 16),device= device),
            layers.Relu(),
            layers.Conv(size=(8, 8), device= device),
            layers.Relu(),
            layers.Conv(size=(4, 4), device= device),
            layers.Relu(),
            layers.Flatten(dim=1),
            layers.Linear(size=(9, 10), device= device),
            layers.Relu(),
            layers.Linear(size=(10, 10), device= device),
            layers.Softmax(n_classes=10),
        ]

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        a = X
        for l in self.layers:
            a = l(a)
        return a

    def backward(self, loss_grad: torch.Tensor):
        grad = loss_grad
        for l in reversed(self.layers):
            grad = l.backward(grad)

In [30]:
from mydllib.losses import CategoricalCrossEntropyLoss
from mydllib.optimizers import BatchGradientDescent

model = MNIST784(device=device)
loss_fn = CategoricalCrossEntropyLoss()
optim = BatchGradientDescent(lr=3e-2)

In [31]:
from mydllib.functional.metrics import accuracy

class BatchLoader:
  def __init__(self, input_data, target_data, batch_size, shuffle=True):
    self.batch_size = batch_size
    self.input_data = input_data
    self.target_data = target_data
    self.shuffle = shuffle
    self.num_samples = len(input_data)
    self.num_batches = (self.num_samples + self.batch_size - 1) // self.batch_size
    self.current_batch = 0
    self.indices = torch.arange(self.num_samples)

    if self.shuffle:
        self.indices = self.indices[torch.randperm(self.num_samples)]

  def __iter__(self):
    return self

  def __next__(self):
    if self.current_batch >= self.num_batches:
        raise StopIteration

    start_idx = self.current_batch * self.batch_size
    end_idx = min((self.current_batch + 1) * self.batch_size, self.num_samples)

    batch_indices = self.indices[start_idx:end_idx]

    batch_input = self.input_data[batch_indices]
    batch_target = self.target_data[batch_indices]

    self.current_batch += 1

    return batch_input, batch_target

# Training proccess
epochs = 15
loss_h = []
tloss_h = []
for epoch in range(epochs):
  loader = BatchLoader(X_train, y_train, batch_size=8192, shuffle=True)
  test_loader = BatchLoader(X_train, y_train, batch_size=8192, shuffle=True)

  total_acc = 0
  total_loss = 0
  total_tacc = 0
  total_tloss = 0

  for batch_x, batch_y in loader:
    # Forward propagation
    y_pred = model(batch_x)
    loss = loss_fn(y_true=batch_y, y_hat=y_pred)
    total_acc += accuracy(batch_y, y_pred)*100 / loader.num_batches
    total_loss += loss / loader.num_batches
    # Backward propagation
    loss_grad = loss_fn.backward(y_true=batch_y, y_hat=y_pred)
    model.backward(loss_grad)
    # Update parameters
    optim.step(model.param_groups())

  for batch_x, batch_y in test_loader:
    # Forward propagation
    with model.cache_preserve():
      y_pred = model(batch_x)
      loss = loss_fn(y_true=batch_y, y_hat=y_pred)
      total_tloss += loss / test_loader.num_batches
      total_tacc += accuracy(batch_y, y_pred)*100 / loader.num_batches

  if epoch % 1 == 0:
    print(
        f"[Epoch: {epoch}/{epochs}] - Loss: {total_loss.item():.6f} - Acc: {total_acc:.2f}%"
        f" | TestLoss: {total_tloss.item():.6f} - TestAcc: {total_tacc:.2f}% ")

[Epoch: 0/15] - Loss: 2.301883 - Acc: 11.65% | TestLoss: 2.300347 - TestAcc: 12.85% 
[Epoch: 1/15] - Loss: 2.299791 - Acc: 12.94% | TestLoss: 2.298769 - TestAcc: 13.14% 
[Epoch: 2/15] - Loss: 2.297684 - Acc: 13.38% | TestLoss: 2.296403 - TestAcc: 13.27% 
[Epoch: 3/15] - Loss: 2.294392 - Acc: 13.38% | TestLoss: 2.291352 - TestAcc: 12.82% 
[Epoch: 4/15] - Loss: 2.286910 - Acc: 12.04% | TestLoss: 2.278740 - TestAcc: 10.64% 
[Epoch: 5/15] - Loss: 2.265138 - Acc: 9.83% | TestLoss: 2.238404 - TestAcc: 9.30% 
[Epoch: 6/15] - Loss: 2.206747 - Acc: 12.04% | TestLoss: 2.167608 - TestAcc: 19.34% 
[Epoch: 7/15] - Loss: 2.140634 - Acc: 21.11% | TestLoss: 2.112246 - TestAcc: 23.04% 
[Epoch: 8/15] - Loss: 2.088416 - Acc: 24.46% | TestLoss: 2.055830 - TestAcc: 25.70% 
[Epoch: 9/15] - Loss: 2.032473 - Acc: 26.53% | TestLoss: 2.000882 - TestAcc: 27.79% 
[Epoch: 10/15] - Loss: 1.979456 - Acc: 28.65% | TestLoss: 1.950516 - TestAcc: 29.73% 
[Epoch: 11/15] - Loss: 1.931337 - Acc: 30.52% | TestLoss: 1.904086